### Face Recognition Using Deep Face

In [1]:
from deepface import DeepFace
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

similarity_threshold = 100
img1_path = "20.jpg"
img2_path = "21.jpg"

result = DeepFace.verify(img1_path, img2_path)

img1 = mpimg.imread(img1_path)
img2 = mpimg.imread(img2_path)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.imshow(img1)
ax1.set_title('Image 1')

ax2.imshow(img2)
ax2.set_title('Image 2')

plt.show()

similarity_percentage = round(result["distance"] * 100, 2)
confidence_score=similarity_threshold - similarity_percentage
if result["verified"]:
    print(f"Both faces are similar.\n Confidence Score is {confidence_score}")
else:
    print(f"Both faces are dissimilar.\n Confidence Score is {confidence_score}")


ValueError: Confirm that 1.jpg exists

### Face Similarity Using Face Net

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN, InceptionResnetV1, prewhiten
from torchvision import transforms
from PIL import Image

# Load FaceNet model
model = InceptionResnetV1(pretrained='vggface2').eval()

# Create MTCNN face detector
mtcnn = MTCNN(keep_all=True, min_face_size=20)

def preprocess_image(image_path):
    img = Image.open(image_path)
    img_cropped = mtcnn(img)
    
    if img_cropped is not None:
        img_cropped = transforms.functional.resize(img_cropped, (160, 160))
        img_cropped = prewhiten(img_cropped)

        if not isinstance(img_cropped, torch.Tensor):
            img_cropped_tensor = transforms.functional.to_tensor(img_cropped)
        else:
            img_cropped_tensor = img_cropped.clone()
        
        return img_cropped_tensor, img
    else:
        return None, None

def calculate_similarity(img1, img2, model):
    with torch.no_grad():
        img1_embedding = model(img1)
        img2_embedding = model(img2)

    distance = F.pairwise_distance(img1_embedding, img2_embedding)
    similarity_percentage = 100 - distance.item()

    return similarity_percentage, img1_embedding.cpu().detach(), img2_embedding.cpu().detach()

# File paths
img1_path = "2.jpg"
img2_path = "3.jpg"

# Preprocess images
img1_tensor, img1_pil = preprocess_image(img1_path)
img2_tensor, img2_pil = preprocess_image(img2_path)

if img1_tensor is not None and img2_tensor is not None:
    # Calculate similarity
    confidence_percentage, _, _ = calculate_similarity(img1_tensor, img2_tensor, model)
    similarity_threshold = 99.0

    # Visualize images
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

    ax1.imshow(img1_pil)
    ax1.set_title('Image 1')

    ax2.imshow(img2_pil)
    ax2.set_title('Image 2')

    plt.show()

    # Display result message
    if confidence_percentage >= similarity_threshold:
        print(f"Images are Similar.\nConfidence score: {confidence_percentage:.2f}%")
    else:
        print(f"Imaeges are Dissimilar.\nConfidence score: {confidence_percentage:.2f}%")
else:
    print("No face detected in one or both of the images.")


### Face Similarity using VGG19

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def calculate_similarity(img1_path, img2_path, model):
    img1_array = preprocess_image(img1_path)
    img2_array = preprocess_image(img2_path)

    img1_embedding = model.predict(img1_array)
    img2_embedding = model.predict(img2_array)

    # Calculate cosine similarity
    similarity_score = np.dot(img1_embedding.flatten(), img2_embedding.flatten()) / (np.linalg.norm(img1_embedding) * np.linalg.norm(img2_embedding))
    similarity_percentage = similarity_score * 100

    return similarity_percentage

# File paths
img1_path = "Ranveer2.jpg"
img2_path = "ranveer.jpg"

# Load VGG19 model
vgg_model = VGG19(weights='imagenet', include_top=False)

# Calculate similarity
similarity_percentage = calculate_similarity(img1_path, img2_path, vgg_model)

# Visualize images
img1 = mpimg.imread(img1_path)
img2 = mpimg.imread(img2_path)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.imshow(img1)
ax1.set_title('Image 1')

ax2.imshow(img2)
ax2.set_title('Image 2')

plt.show()

# Display result message
print("Are these faces the same person?", similarity_percentage > 50)
print(f"Similarity score: {similarity_percentage:.2f}%")


### Face Similarity Using VGG16

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def calculate_similarity(img1_path, img2_path, model):
    img1_array = preprocess_image(img1_path)
    img2_array = preprocess_image(img2_path)

    # Get VGG16 embeddings
    img1_embedding = model.predict(img1_array)
    img2_embedding = model.predict(img2_array)

    # Calculate cosine similarity
    similarity_score = np.dot(img1_embedding.flatten(), img2_embedding.flatten()) / (np.linalg.norm(img1_embedding) * np.linalg.norm(img2_embedding))
    similarity_percentage = similarity_score * 100

    return similarity_percentage

# File paths
img1_path = "Ranveer2.jpg"
img2_path = "ranveer.jpg"

# Load VGG16 model
vgg_model = VGG16(weights='imagenet', include_top=False)

# Calculate similarity
similarity_percentage = calculate_similarity(img1_path, img2_path, vgg_model)

# Visualize images
img1 = mpimg.imread(img1_path)
img2 = mpimg.imread(img2_path)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.imshow(img1)
ax1.set_title('Image 1')

ax2.imshow(img2)
ax2.set_title('Image 2')

plt.show()

# Display result message
print("Are these faces the same person?", similarity_percentage > 50)
print(f"Similarity score: {similarity_percentage:.2f}%")


### Face Similarity Model Using Resnet

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

similarity_threshold = 100

# Load pre-trained ResNet model
resnet_model = ResNet50(weights='imagenet', include_top=False)

def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array

def calculate_similarity(img1_path, img2_path, model):
    img1_array = preprocess_image(img1_path)
    img2_array = preprocess_image(img2_path)

    img1_embedding = model.predict(img1_array)
    img2_embedding = model.predict(img2_array)

    # Calculate cosine similarity
    similarity_score = np.dot(img1_embedding.flatten(), img2_embedding.flatten()) / (np.linalg.norm(img1_embedding) * np.linalg.norm(img2_embedding))
    similarity_percentage = similarity_score * 100

    return similarity_percentage

# File paths
img1_path = "3.jpg"
img2_path = "4.jpg"

# Calculate similarity
similarity_percentage = calculate_similarity(img1_path, img2_path, resnet_model)

# Visualize images
img1 = mpimg.imread(img1_path)
img2 = mpimg.imread(img2_path)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))

ax1.imshow(img1)
ax1.set_title('Image 1')

ax2.imshow(img2)
ax2.set_title('Image 2')

plt.show()

# Display result message
confidence_score = similarity_threshold - similarity_percentage
if similarity_percentage > similarity_threshold:
    print(f"Both faces are similar.\nScore is {confidence_score}")
else:
    print(f"Both faces are dissimilar.\nScore is {confidence_score}")
